# **Vectorizing and Clustering**

imports

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import os
import json
import hdbscan
# Level up one level directory to add app the the allowed routes
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from app.utils.base_dir import BASE_DIR


## **1. Vectorizing**

open the json and load the text and label

In [9]:


path = BASE_DIR/'data'/'processed'/'processedData.json' 
with open(path, 'r') as f:
    docs = json.load(f)

label = [list(d.keys())[0] for d in docs]
text = [list(d.values())[0] for d in docs]


check if is the text is none

In [10]:
for i, doc in enumerate(text):
    if doc is None:
        print(f"Documento {i} es None")
    elif not isinstance(doc, str):
        print(f"Documento {i} no es str: {type(doc)} -> {repr(doc)}")
    elif not doc.strip():
        print(f"Documento {i} está vacío")


### **1.1 Vectorizing the text with TF_IDF**

### **1.1 Vectorizing the text with TF_IDF**

In [ ]:

def TF_IDF_vectorizer(text_corpus: list):
    vectorizer = TfidfVectorizer(
        #max_features= 10000,  # the max num of words for the total corpus 
        max_df=0.95,    #delete if the word apears on the 85% of docs
        min_df=0.001,    #delete if tye word aperas just on the 0.1% of docs
    )
    return vectorizer.fit_transform(text_corpus)
TF_vector = TF_IDF_vectorizer(text)
print(TF_vector) 

## **2. Clustering**

### **HDBSACAN**
More strict than k means
- The documments need to be very closer

In [24]:
def cluster_hdbscan(TF_vector):
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=3,
        min_samples=1
    )
    # return a object like [0,1,5,0,1,5,6,9,0] where every number in order is the number of cluster of the document
    return clusterer.fit_predict(TF_vector)
    
print(cluster_hdbscan(TF_vector))


[-1 43 41 -1 -1 40 43 -1 -1 -1 -1  4 -1 -1 48 -1 -1 -1 -1 -1 -1 -1 44 39
 -1  3 -1 -1 33 23 -1 -1 -1 -1 -1 -1  0  0  0  2  2 -1  2 42 -1 39 -1 43
 -1  1  1  1 -1 45 48 -1 45 45 -1 -1 -1 -1 -1 -1 38 -1 -1 -1 -1 18 -1 -1
 18  5  5  5  5 -1 -1  5 -1 15 15 -1 33 39 33 -1 -1 -1 33 33 37 -1 30 30
 -1 -1 -1 -1 -1 32 -1 32  1  1 -1 22 34 -1 34 22 22 34 -1 -1 -1  4 47  4
 -1 -1 -1 -1 47 17 -1 -1  9 -1  9  9 -1 -1 -1 23 -1 25 -1 46 46 46 13 -1
 -1 47 -1 -1 47 43 43 19 10 43 26 -1 26 -1  7 -1  7 23  7 -1  7  7 44 -1
 -1 39 -1 -1 -1 -1 43 -1 -1 43 23 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1
 -1 -1 -1 36 -1 -1 36 -1 39 -1 -1 -1 43 -1 10 10 -1 47 47 47 47 47 -1 47
 -1 -1 -1 -1 12 -1 -1 -1 -1 -1 -1 -1 -1 32 32 32 32 40 32 32 -1 33 -1 14
 41 -1 -1 41 35 -1 41 41 41 -1 33 41 -1 -1 33 -1 -1 16 -1 -1 46 -1 -1 -1
  8  8  8  8 21 35 46 31 31 31 31 -1 48  3 -1 -1 -1 -1 -1 -1 24 -1 -1 -1
 23 15  3 -1 24 24 -1 -1 18 -1 41 41 -1 46 -1 -1 47 10 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 39 44 -1 40 -1 -1 -1 47